1. Import Library

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

2. Dataset read

In [7]:
train_df = pd.read_csv("./dataset/new_train.csv")
test_df = pd.read_csv("./dataset/new_test.csv")

3. Dataset Shape

In [8]:
print("DATASET SHAPE:", train_df.shape)
print(train_df.head())

DATASET SHAPE: (32950, 16)
   age           job   marital          education  default housing loan  \
0   49   blue-collar   married           basic.9y  unknown      no   no   
1   37  entrepreneur   married  university.degree       no      no   no   
2   78       retired   married           basic.4y       no      no   no   
3   36        admin.   married  university.degree       no     yes   no   
4   59       retired  divorced  university.degree       no      no   no   

     contact month day_of_week  duration  campaign  pdays  previous  \
0   cellular   nov         wed       227         4    999         0   
1  telephone   nov         wed       202         2    999         1   
2   cellular   jul         mon      1148         1    999         0   
3  telephone   may         mon       120         2    999         0   
4   cellular   jun         tue       368         2    999         0   

      poutcome    y  
0  nonexistent   no  
1      failure   no  
2  nonexistent  yes  
3  none

4. Separate input & target

In [27]:
# convert to 0/1
X = train_df.drop("y", axis=1)
y = train_df["y"].map({"yes": 1, "no": 0})

5. Preprocessing Pipeline

In [28]:
categorical_cols = ['job', 'marital', 'education', 'default',
                    'housing', 'loan', 'contact', 'month',
                    'day_of_week', 'poutcome']

numeric_cols = ['age', 'duration', 'campaign', 'pdays', 'previous']

6. Preprocessing pipeline

In [29]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
        ('num', StandardScaler(), numeric_cols)
    ])

7. Transform data

In [31]:
X_processed = preprocessor.fit_transform(X).toarray()

8. Train-test split

In [32]:
X_train, X_val, y_train, y_val = train_test_split(
    X_processed, y, test_size=0.2, random_state=42)

9. Build ANN

In [33]:
import tensorflow as tf
from tensorflow.keras import layers, models

# binary classification
ann = models.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')   
])

ann.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

ann.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                3776      
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 5,889
Trainable params: 5,889
Non-trainable params: 0
_________________________________________________________________


10. Train ANN

In [34]:
history_ann = ann.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32
)

Epoch 1/20
824/824 [==============================] - 4s 4ms/step - loss: 0.2331 - accuracy: 0.9014 - val_loss: 0.2228 - val_accuracy: 0.9023
Epoch 2/20
824/824 [==============================] - 3s 3ms/step - loss: 0.2062 - accuracy: 0.9081 - val_loss: 0.2201 - val_accuracy: 0.9052
Epoch 3/20
824/824 [==============================] - 3s 3ms/step - loss: 0.1999 - accuracy: 0.9105 - val_loss: 0.2146 - val_accuracy: 0.9041
Epoch 4/20
824/824 [==============================] - 3s 3ms/step - loss: 0.1948 - accuracy: 0.9125 - val_loss: 0.2123 - val_accuracy: 0.9015
Epoch 5/20
824/824 [==============================] - 3s 3ms/step - loss: 0.1909 - accuracy: 0.9153 - val_loss: 0.2106 - val_accuracy: 0.9011
Epoch 6/20
824/824 [==============================] - 3s 3ms/step - loss: 0.1863 - accuracy: 0.9154 - val_loss: 0.2119 - val_accuracy: 0.9018
Epoch 7/20
824/824 [==============================] - 3s 3ms/step - loss: 0.1833 - accuracy: 0.9182 - val_loss: 0.2161 - val_accuracy: 0.9052
Epoch 

11. CNN MODEL